In [26]:
# load the model
import torch
from mttl.utils import get_checkpoint_path
from mttl.models.t0_encoder_decoder import T0EncoderDecoder
chekpoint = "pretrain_tensororderpoly_lora_order_4"
ckpt_path = get_checkpoint_path(chekpoint)

ckpt = torch.load(ckpt_path)




3 checkpoints found. taking the one with the lowest val loss


Found checkpoint pretrain_tensororderpoly_lora_order_4/global_step100000.pt


In [27]:
from mttl.config import Config

config = Config("t0/finetune.json+t0/copa.json")
print(config.finetune_task_name)
ckpt_dict = ckpt["state_dict"]
ckpt_args = ckpt["hyper_parameters"]
config.old_exp_name = ckpt_args["exp_name"]

ARGS_TO_OVERWRITE = [
    "dataset",
    "config",
    "finegrained",
    "lora_rank",
    "max_input_length",
    "max_output_length",
    "model",
    "n_skills",
    "n_splits",
    "n_tasks",
    "use_t0_templates_as_tasks",
    "module_logits_relaxed_bernoulli",
    "module_logits_straight_through",
    "poly_use_shared_skill",
    "poly_average_correction",
    "poly_selector",
    "lora_rank",
    "lora_modules",
    "lora_layers",
    "model_modifier",
    "use_task_descriptions",
    "trainable_param_names",
    "use_precomputed_task_embeddings",
    "num_pos_examples",
    "example_to_ids_path",
]

for arg_name in ARGS_TO_OVERWRITE:
    # we over-write with a new one only if the argument was a default one
    if arg_name in ckpt_args and not config.was_overridden(arg_name):
        print("Overwriting", arg_name, "=", ckpt_args[arg_name])
        setattr(config, arg_name, ckpt_args[arg_name])

copa
Overwriting lora_rank = 16
Overwriting max_input_length = 256
Overwriting max_output_length = 256
Overwriting model = bigscience/T0_3B
Overwriting n_skills = 8
Overwriting n_splits = 4
Overwriting n_tasks = 313
Overwriting use_t0_templates_as_tasks = True
Overwriting module_logits_relaxed_bernoulli = False
Overwriting module_logits_straight_through = False
Overwriting poly_use_shared_skill = False
Overwriting poly_average_correction = False
Overwriting poly_selector = poly
Overwriting lora_rank = 16
Overwriting lora_modules = .*SelfAttention|.*EncDecAttention|.*DenseReluDense
Overwriting lora_layers = q|k|v|o|wi_1.*|wi_0.*|wo
Overwriting model_modifier = tensororderpoly_lora
Overwriting use_task_descriptions = False
Overwriting trainable_param_names = .*weight_leafs_[ab].*|.*module_logits.*
Overwriting num_pos_examples = 0
Overwriting example_to_ids_path = None


In [28]:
from mttl.datamodule.t0_data_module import T0FinetuneDataModule
dm = T0FinetuneDataModule(config)
module = T0EncoderDecoder(**vars(config),tokenizer=dm.tokenizer)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
created 432 for a total of 432 adapted layers
Registering method:  get_adapters
Registering method:  get_selectors
Registering method:  resize_module_logits
Registering method:  switch_selector_to_average
T5Block(
  (layer): ModuleList(
    (0): T5LayerSelfAttention(
      (SelfAttention): T5Attention(
        (q): PolyLoRATensorOrder(
          (selector): PolytroponSelector()
          (layerone_normalization_a): LayerNorm((16, 49), eps=1e-05, elementwise_affine=True)
          (layertwo_normalization_a): LayerNorm((16, 49), eps=1e-05, elementwise_affine=True)
          (layerone_normalization_b): LayerNorm((16, 49), eps=1e-05, elementwise_affine=True)
          (layertwo_normalization_b): LayerNorm((16, 49), eps=1e-05, elementwise_affine=True)
        )
        (k): PolyLoRATensorOrder(
          (selector): PolytroponSelector()
          (layerone_normalization_a): LayerNorm((16, 49), eps=1e-05, elementwi

In [29]:
module.load_state_dict(ckpt_dict, strict=False)

Loading keys... ['model.encoder.block.0.layer.0.SelfAttention.q.weight_leafs_a', 'model.encoder.block.0.layer.0.SelfAttention.q.weight_leafs_b', 'model.encoder.block.0.layer.0.SelfAttention.q.selector.module_logits', 'model.encoder.block.0.layer.0.SelfAttention.k.weight_leafs_a', 'model.encoder.block.0.layer.0.SelfAttention.k.weight_leafs_b', 'model.encoder.block.0.layer.0.SelfAttention.k.selector.module_logits', 'model.encoder.block.0.layer.0.SelfAttention.v.weight_leafs_a', 'model.encoder.block.0.layer.0.SelfAttention.v.weight_leafs_b', 'model.encoder.block.0.layer.0.SelfAttention.v.selector.module_logits', 'model.encoder.block.0.layer.0.SelfAttention.o.weight_leafs_a', 'model.encoder.block.0.layer.0.SelfAttention.o.weight_leafs_b', 'model.encoder.block.0.layer.0.SelfAttention.o.selector.module_logits', 'model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight_leafs_a', 'model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight_leafs_b', 'model.encoder.block.0.layer.1.DenseReluDense.w

_IncompatibleKeys(missing_keys=['model.shared.weight', 'model.encoder.embed_tokens.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.layerone_normalization_a.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.layerone_normalization_a.bias', 'model.encoder.block.0.layer.0.SelfAttention.q.layertwo_normalization_a.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.layertwo_normalization_a.bias', 'model.encoder.block.0.layer.0.SelfAttention.q.layerone_normalization_b.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.layerone_normalization_b.bias', 'model.encoder.block.0.layer.0.SelfAttention.q.layertwo_normalization_b.weight', 'model.encoder.block.0.layer.0.SelfAttention.q.layertwo_normalization_b.bias', 'model.encoder.block.0.layer.0.SelfAttention.k.weight', 'model.encoder.block.0.layer.0.SelfAttention.k.layerone_normalization_a.weight', 'model.encoder.block.0.layer.0.SelfAttention.k.layerone_normalization_a.bia

In [31]:
module.model.decoder.block[0].layer[0].SelfAttention.k.weight_leafs_a.shape

torch.Size([4, 8, 16, 7])